In [1]:
import gc
import os

import polars as pl
import torch
from tqdm.notebook import tqdm

# import warnings
# warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
if torch.cuda.is_available():
    print('device name:', torch.cuda.get_device_name(device))

EVAL_BATCH_SIZE = 128

def collect(*, verbose=True):
    if verbose:
        print('garbage collector collected %d objects' % gc.collect())
    else:
        gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

collect()

device: cuda
device name: NVIDIA GeForce RTX 3050 Laptop GPU
garbage collector collected 56 objects


In [2]:
news = pl.read_parquet('private-data/filtered_pairs_2022.parquet')

print(len(news), 'rows')

news.head(3)

5946 rows


pair_id,summary_id,article_id,summary,article,simhash_distance,summary_title,article_title
u32,i32,i32,str,str,i64,str,str
1093692,43423102,43422899,"""Tại phiên họp thứ 14 ngày 11/8…","""100% Ủy viên Ủy ban Thường vụ …",16,"""Quốc hội thông qua việc thành …","""Thành lập thị xã Chơn Thành th…"
1718596,43370015,43360018,"""Tập đoàn Điện lực Việt Nam (EV…","""Theo thông tin từ Tập đoàn Điệ…",17,"""Xuất hiện trang web giả mạo th…","""Xuất hiện trang web giả mạo th…"
1258448,44012345,44017658,"""Mưa lớn vừa qua khiến hàng tấn…","""Trận mưa lớn, liên tục trong c…",14,"""Cảnh tan hoang ở nghĩa trang l…","""Xót xa cảnh nghìn ngôi mộ tại …"


In [3]:
from bert_score import score as bert_score

collect()

samples = news.sample(3, seed=2025).select('summary', 'article')

for i, model_name in enumerate(tqdm([
    'bert-base-multilingual-cased',
    # 'google/mt5-small',
    # 'google/mt5-base',
    # 'google/mt5-large',
    # 'google/mt5-xl',
    'facebook/mbart-large-50-many-to-many-mmt',
])):
    precisions, recalls, _ = bert_score(
        samples.select('summary').to_series().to_list(),
        samples.select('article').to_series().to_list(),
        model_type=model_name,
        lang='vi',
        use_fast_tokenizer=True,
        idf=False, 
        batch_size=min(EVAL_BATCH_SIZE, len(samples)),
        verbose=False, 
        device=device,
    )
    bs = {
        'bs_p': precisions.tolist(),
        'bs_r': recalls.tolist(),
    }
    print('🌱 bs', bs)
    for score_name, score_values in bs.items():
        samples = samples.with_columns(
            pl
            .Series(score_values)
            .alias(score_name + '_' + str(i))
        )

samples

garbage collector collected 0 objects


  0%|          | 0/2 [00:00<?, ?it/s]

🌱 padded tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
⚜️ i 0
⚜️ lens[i] tensor(512)
⚜️ a [101, 103843, 10376, 126, 118, 10186, 117, 23158, 31352, 18327, 117, 21631, 90349, 30355, 14549, 61697, 37489, 21217, 16244, 31701, 40352, 20005, 14369, 32692, 23270, 117, 11125, 18868, 117, 82378, 15633, 30097, 72959, 69014, 27016, 31701, 40352, 18581, 14938, 127, 117, 21631, 33939, 53626, 13910, 14549, 61697, 37489, 68123, 13892, 117, 15991, 15054, 19154, 34270, 16948, 16886, 16117, 34270, 38122, 119, 298, 10116, 91659, 19168, 14549, 61697, 12086, 31701, 16188, 12944, 20270, 39935, 117, 30617, 14755, 16425, 117, 11629, 16851, 21727, 36921, 16886, 50429, 11182, 14789, 10150, 119, 10259, 15509, 19168, 18928, 15222, 117, 18928, 13388, 12598, 20005, 14369, 11182, 14789, 10417, 31450, 23664, 10115, 13848, 117, 64630, 15202, 20089, 13223, 32692, 23270, 117, 11125, 18868, 119, 141, 42397, 16425, 40352, 10601, 51924, 15202, 23158, 31352, 18327, 11

summary,article,bs_p_0,bs_r_0,bs_p_1,bs_r_1
str,str,f64,f64,f64,f64
"""Khi đang lẩn trốn tại một địa …","""Sau hai ngày trốn khỏi trại gi…",0.799396,0.691559,0.990648,0.987232
"""Cơ quan công an đang điều tra …","""Nguyên nhân vụ việc ban đầu đư…",0.783248,0.624593,0.986422,0.973143
"""Sáng 5/12, Bộ Chính trị, Ban B…","""Sáng 5-12, Bộ Chính trị, Ban B…",0.800523,0.608533,0.981096,0.985774
